In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Install required packages
!pip install transformers==4.52.4
!pip install sentence-transformers
!pip install PyPDF2
!pip install faiss-cpu
!pip install streamlit
!pip install pyngrok
!pip install python-multipart
!pip install langchain
!pip install google-generativeai
!pip install google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 84.0 MB/s eta 0:00:00:00:01:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━

In [7]:
%%writefile app.py
import streamlit as st
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from PyPDF2 import PdfReader
import faiss
import torch
import time
import warnings
import tempfile
import os
import re
from io import BytesIO
from langchain.text_splitter import RecursiveCharacterTextSplitter

warnings.filterwarnings('ignore')

# Set page config
st.set_page_config(
    page_title="Chat with PDF - Enhanced Version",
    page_icon="📚",
    layout="wide"
)

class PDFQASystem:
    def __init__(self):
        self.embedding_model = None
        self.index = None
        self.chunks = None
        self.initialized = False
        self.pdf_name = None
        self.full_text = ""
        self.llm_client = None
        self.api_key = None
    
    def set_api_key(self, api_key):
        """Set API key"""
        self.api_key = api_key
        if api_key:
            try:
                from google import genai
                self.llm_client = genai.Client(api_key=api_key)
                return True
            except Exception as e:
                st.error(f"❌ Error initializing Gemini: {e}")
                return False
        return False
    
    def extract_text_from_pdf(self, pdf_file):
        """Extract text from PDF file"""
        try:
            pdf_reader = PdfReader(pdf_file)
            full_text = ""
            
            for page in pdf_reader.pages:
                text = page.extract_text()
                if text:
                    full_text += text + "\n"
            
            self.full_text = full_text
            return full_text
        except Exception as e:
            st.error(f"Error reading PDF: {e}")
            return None
    
    def chunk_text(self, text, chunk_size=800, overlap=100):
        """Split text into chunks using LangChain"""
        try:
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=chunk_size,
                chunk_overlap=overlap,
                separators=["\n\n", "\n", ". ", "! ", "? ", " ", ""]
            )
            
            chunks = text_splitter.split_text(text)
            return chunks
        except Exception as e:
            st.warning(f"Using basic chunking method: {e}")
            sentences = text.split('. ')
            chunks = []
            current_chunk = ""
            
            for sentence in sentences:
                if len(current_chunk) + len(sentence) < chunk_size:
                    current_chunk += sentence + '. '
                else:
                    if current_chunk:
                        chunks.append(current_chunk.strip())
                    current_chunk = sentence + '. '
            
            if current_chunk:
                chunks.append(current_chunk.strip())
            
            return chunks
    
    def embed_chunks(self, chunks):
        """Create embeddings for text chunks using multilingual model"""
        model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
        
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        model = model.to(device)
        
        embeddings = model.encode(chunks, convert_to_numpy=True)
        return model, embeddings
    
    def create_faiss_index(self, embeddings):
        """Create FAISS index for semantic search"""
        dim = embeddings.shape[1]
        index = faiss.IndexFlatL2(dim)
        index.add(embeddings)
        return index
    
    def search_index(self, query, k=5):
        """Search for relevant chunks using semantic similarity"""
        query_embedding = self.embedding_model.encode([query], convert_to_numpy=True)
        distances, indices = self.index.search(query_embedding, k)
        
        relevant_chunks = []
        for i, idx in enumerate(indices[0]):
            if idx < len(self.chunks):
                relevant_chunks.append(self.chunks[idx])
        
        return relevant_chunks
    
    def initialize_system(self, pdf_file, pdf_name):
        """Initialize the Q&A system with uploaded PDF"""
        try:
            text = self.extract_text_from_pdf(pdf_file)
            
            if text is None or len(text.strip()) == 0:
                st.error("Could not extract text from PDF. The file might be scanned or corrupted.")
                return False
            
            with st.spinner("🔄 Splitting text into chunks..."):
                self.chunks = self.chunk_text(text)
            
            with st.spinner("🧠 Creating embeddings..."):
                self.embedding_model, embeddings = self.embed_chunks(self.chunks)
            
            with st.spinner("🔍 Building search index..."):
                self.index = self.create_faiss_index(embeddings)
            
            self.initialized = True
            self.pdf_name = pdf_name
            return True
            
        except Exception as e:
            st.error(f"Error during initialization: {str(e)}")
            return False
    
    def _generate_llm_answer(self, question, context):
        """Generate coherent answer using large language model"""
        if self.llm_client is None:
            clean_answer = self._create_fallback_answer(context).replace('\\n', '\n').replace('\n\n', '\n')
            return f"💡 **Answer from text search:**\n\n{clean_answer}"
        
        try:
            prompt = f"""
            You are an intelligent assistant specialized in analyzing documents and answering questions based on the information in them.
            
            Task:
            - Read the provided context carefully
            - Answer the question based only on the information in the context
            - If information is insufficient, state that clearly
            - Use clear and understandable language
            - Structure the answer logically
            
            Question: {question}
            
            Context from document:
            {context}
            
            Answer:
            """
            
            response = self.llm_client.models.generate_content(
                model='gemini-2.0-flash-exp',
                contents=prompt
            )
            
            clean_response = response.text.replace('\\n', '\n').replace('\n\n', '\n')
            return f"🤖 **Smart answer from language model:**\n\n{clean_response}"
            
        except Exception as e:
            clean_answer = self._create_fallback_answer(context).replace('\\n', '\n').replace('\n\n', '\n')
            return f"💡 **Answer from text search:**\n\n{clean_answer}"
    
    def _create_fallback_answer(self, context):
        """Create alternative answer when LLM is not available"""
        clean_context = context.replace('\\n', ' ').replace('\n', ' ')
        sentences = clean_context.split('. ')
        if len(sentences) > 4:
            return ". ".join(sentences[:4]) + "..."
        else:
            return clean_context
    
    def answer_question(self, question):
        """Generate answer for a given question"""
        if not self.initialized:
            return "System not initialized. Please upload a PDF file first.", []
        
        try:
            top_chunks = self.search_index(question, k=5)
            context = " ".join(top_chunks)
            
            if not context.strip():
                return "❌ Could not find relevant information in the document.", []
            
            answer = self._generate_llm_answer(question, context)
            
            return answer, top_chunks
            
        except Exception as e:
            return f"Error generating answer: {str(e)}", []

@st.cache_resource
def load_qa_system():
    return PDFQASystem()

def main():
    st.title("📚 Chat with PDF - Enhanced Version")
    st.markdown("### Upload a PDF and ask questions about its content")
    
    qa_system = load_qa_system()
    
    with st.sidebar:
        st.markdown("### 🔑 AI Settings (Optional)")
        
        api_key = st.text_input(
            "Google AI API Key (Optional)",
            type="password",
            placeholder="Enter API key to activate AI...",
            help="Get free key from: https://aistudio.google.com/app/apikey"
        )
        
        if api_key:
            if st.button("Activate AI"):
                if qa_system.set_api_key(api_key):
                    st.success("✅ AI activated successfully!")
                else:
                    st.error("❌ Failed to activate AI. Check your key.")
        
        st.markdown("---")
        st.markdown("### 📄 Upload PDF")
        uploaded_file = st.file_uploader(
            "Choose PDF file",
            type="pdf",
            help="Upload a PDF document to chat with"
        )
        
        if uploaded_file is not None:
            with st.spinner("Processing PDF..."):
                success = qa_system.initialize_system(uploaded_file, uploaded_file.name)
                
                if success:
                    st.success(f"✅ PDF processed successfully!")
                    st.info(f"**File:** {uploaded_file.name}")
                    st.info(f"**Chunks processed:** {len(qa_system.chunks)}")
                    
                    text_sample = qa_system.chunks[0][:300] + "..." if qa_system.chunks else "No text extracted"
                    with st.expander("📊 Document Preview"):
                        st.text_area("First chunk sample:", text_sample, height=120)
                else:
                    st.error("Failed to process PDF. Please try another file.")
        
        st.markdown("---")
        st.markdown("### 💡 Tips for Better Answers")
        st.markdown("""
        **Without API Key:**
        - System will rely on precise text search
        - Answers will be directly from the document
        
        **With API Key:**
        - Smarter and more coherent answers
        - Better context understanding
        - Information summarization
        """)
        
        st.markdown("---")
        st.markdown("### 🛠️ System Information")
        st.info(f"""
        **Models Used:**
        - Embedding: multilingual-mpnet-base-v2
        - AI: {'Activated' if qa_system.llm_client else 'Not activated'}
        - Search: FAISS
        - Chunking: LangChain
        """)
    
    col1, col2 = st.columns([2, 1])
    
    with col1:
        st.markdown("### 💬 Chat")
        
        if "messages" not in st.session_state:
            st.session_state.messages = []
        
        for message in st.session_state.messages:
            with st.chat_message(message["role"]):
                st.markdown(message["content"])
                
                if message["role"] == "assistant" and "sources" in message:
                    with st.expander("🔍 View Sources"):
                        for i, source in enumerate(message["sources"], 1):
                            st.markdown(f"**Source {i}:**")
                            st.text(source[:400] + "..." if len(source) > 400 else source)
                            st.markdown("---")
        
        if prompt := st.chat_input("Ask a question about your PDF..."):
            st.session_state.messages.append({"role": "user", "content": prompt})
            
            with st.chat_message("user"):
                st.markdown(prompt)
            
            if not qa_system.initialized:
                response = "Please upload a PDF file first to start chatting."
                sources = []
            else:
                with st.chat_message("assistant"):
                    with st.spinner("🔍 Searching document..."):
                        start_time = time.time()
                        response, sources = qa_system.answer_question(prompt)
                        response_time = time.time() - start_time
                    
                    st.markdown(response)
                    st.caption(f"⏱️ Response time: {response_time:.2f}s")
                    
                    if sources and "Could not find" not in response and "❌" not in response:
                        with st.expander("🔍 View Sources"):
                            for i, source in enumerate(sources, 1):
                                st.markdown(f"**Source {i}:**")
                                st.text(source[:400] + "..." if len(source) > 400 else source)
                                st.markdown("---")
            
            st.session_state.messages.append({
                "role": "assistant", 
                "content": response,
                "sources": sources
            })
    
    with col2:
        st.markdown("### 🎯 Quick Actions")
        
        if st.button("🔄 Clear Chat", use_container_width=True):
            st.session_state.messages = []
            st.rerun()
        
        if st.button("📊 Document Info", use_container_width=True) and qa_system.initialized:
            st.info(f"""
            **Document Analysis:**
            - PDF: {qa_system.pdf_name}
            - Text Chunks: {len(qa_system.chunks)}
            - Search Engine: FAISS
            - Embedding Model: Multilingual
            - AI: {'Activated 🟢' if qa_system.llm_client else 'Not activated 🔴'}
            """)
        
        st.markdown("---")
        st.markdown("### 🚀 Sample Questions")
        st.markdown("""
        **Financial/Budget:**
        - "What is the research budget?"
        - "How much funding for research?"
        
        **Strategic:**
        - "What are the main priorities?"
        - "Strategic focus areas?"
        
        **General:**
        - "What is the main objective?"
        - "Explain the key points"
        """)

if __name__ == "__main__":
    main()

Overwriting app.py


In [8]:
%%writefile deploy.py
import subprocess
import threading
import time
from pyngrok import ngrok
import os

NGROK_AUTH_TOKEN = "35WtIZpirdsJRc5i4LJRatmJ9JP_2kujDdW25KB7R6YyoiKhd"

def run_streamlit():
    subprocess.run(["streamlit", "run", "app.py", "--server.port", "8502", "--server.address", "0.0.0.0"])

def deploy_with_ngrok():
    print("🔐 Setting up ngrok...")
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    
    print("🚀 Starting application...")
    streamlit_thread = threading.Thread(target=run_streamlit)
    streamlit_thread.daemon = True
    streamlit_thread.start()
    
    time.sleep(8)
    
    try:
        public_url = ngrok.connect(8502, bind_tls=True)
        print(f"✅ Application is running at: {public_url}")
        print("💡 Share this link to access the application!")
        
        try:
            while True:
                time.sleep(10)
        except KeyboardInterrupt:
            print("🛑 Shutting down...")
            ngrok.kill()
            
    except Exception as e:
        print(f"❌ Error: {e}")

if __name__ == "__main__":
    deploy_with_ngrok()

Overwriting deploy.py


In [9]:
# Run the deployment
!python deploy.py

🔐 Setting up ngrok...
🚀 Starting application...



  You can now view your Streamlit app in your browser.

  URL: http://0.0.0.0:8502

✅ Application is running at: NgrokTunnel: "https://unchevroned-unprofited-cherie.ngrok-free.dev" -> "http://localhost:8502"
💡 Share this link to access the application!
2025-11-24 11:01:00.148023: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763982060.177842     316 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763982060.186714     316 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
^C
🛑 Shutting down...
  Stopping...
